In [1]:
import os
import pandas as pd
from zipfile import ZipFile

# import fastbook
# from fastbook import *
# from fastai.vision.all import *
# from fastai.vision.widgets import *
# from fastai.metrics import *
# import PIL

In [2]:
data_dir = os.getcwd() + '/data/'
image_dir = os.getcwd() + "/train_images_128pix/"

### Unzip images

In [ ]:
# # Unzip image files to working dir
# fname = 'train_images_128pix.zip'

# with ZipFile(data_dir + fname, 'r') as zip:
#     zip.printdir()
#     zip.extractall()

### Create 'size' field: Train

In [3]:
meta = pd.read_csv(data_dir + "train_metadata.tsv", sep = "\t")

images_128 = meta['image_name']

sizes = []
for image in images_128:
    size = os.stat(image_dir + image).st_size
    sizes.append(size)
    
meta['size'] = sizes
meta.head()

,SDSS_ID,M/L,L_g,distance_Mpc,galsize_kpc,image_name,size_128,size_69,size
0,1237668271371976941,3.022229,1.205884e+10,189.000006,28.852800,1237668271371976941.png,11357,3724,11357
1,1237661873478828168,2.057684,2.953450e+10,201.000002,36.891150,1237661873478828168.png,11705,3748,11705
2,1237651495756234805,5.380972,1.078647e+10,180.428576,33.115510,1237651495756234805.png,11861,3803,11861
3,1237661813339848849,1.525518,1.144418e+10,204.000005,27.124161,1237661813339848849.png,11292,3778,11292
4,1237665532260450879,0.663993,5.546587e+09,208.285712,24.682299,1237665532260450879.png,10870,3733,10870


In [4]:
meta.columns

Index(['SDSS_ID', 'M/L', 'L_g', 'distance_Mpc', 'galsize_kpc', 'image_name',
       'size_128', 'size_69', 'size'],
      dtype='object')

In [4]:
meta.to_csv(data_dir + "train_metadata.tsv", sep = "\t", index = False)

### Create 'size' field: Test

In [5]:
# Load in the eval_69 set
eval_image_dir = os.getcwd() + "/evaluation_images_128pix/"

eval_df = pd.read_csv(data_dir + "eval_metadata.tsv", sep = "\t")
eval_df.head()

,SDSS_ID,L_g,distance_Mpc,galsize_kpc,image_name,size
0,1237657873256349729,2.592800e+10,174.000008,37.521085,1237657873256349729.png,3720
1,1237671260128411724,3.299057e+10,231.857140,33.032891,1237671260128411724.png,3727
2,1237658205036740751,4.094620e+10,310.285730,41.256079,1237658205036740751.png,3773
3,1237663786884006118,8.133352e+09,177.428573,39.151614,1237663786884006118.png,3742
4,1237657594612875451,6.650563e+08,55.714287,7.239235,1237657594612875451.png,3739


In [ ]:
eval_meta = pd.read_csv(data_dir + "eval_metadata.tsv", sep = "\t")

eval_images_128 = eval_df['image_name']

eval_sizes = []
for image in eval_images_128:
    size = os.stat(eval_image_dir + image).st_size
    eval_sizes.append(size)
    
eval_meta['size'] = eval_sizes
eval_meta.head()

In [ ]:
eval_meta.to_csv(data_dir + "eval_metadata.tsv", sep = "\t", index = False)

### Build Dataloader

In [ ]:
img_files = get_image_files(image_dir)
im = PILImage.create(img_files[0])
print(im.shape)
im.to_thumb(160)

In [ ]:
meta['Path'] = image_dir + meta['image_name']
meta.head()

dls = DataBlock(
    blocks=(ImageBlock, RegressionBlock),
    get_x=ColReader('Path', pref='', suff=''), 
    get_y=ColReader('M/L'),
    splitter=RandomSplitter(valid_pct=0.2, seed=42),
    batch_tfms=[*aug_transforms(size=(240,240)), 
                Normalize.from_stats(*imagenet_stats)]
).dataloaders(meta)

In [ ]:
dls.show_batch(max_n=9, figsize=(8,6))

In [ ]:
xb,yb = dls.one_batch()
xb.shape,yb.shape

In [ ]:
min_val = meta['M/L'].min()
max_val = meta['M/L'].max()
learn = cnn_learner(dls, resnet18, y_range=(min_val, max_val))

In [ ]:
dls.loss_func

In [ ]:
#learn.lr_find()

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(min_val, max_val))

lr = 0.08
learn.fine_tune(3, lr)

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(min_val, max_val))
lr = 1e-3
learn.fine_tune(3, lr)

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(min_val, max_val))
lr = 7e-4
learn.fine_tune(3, lr)

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(min_val, max_val))
lr = 5e-4
learn.fine_tune(3, lr)

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(min_val, max_val))

lr = 1e-4
learn.fine_tune(3, lr)

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(min_val, max_val))

lr = 5e-5
learn.fine_tune(3, lr)

In [ ]:
learn = cnn_learner(dls, resnet18, y_range=(min_val, max_val))

lr_steep=1e-5
learn.fine_tune(3, lr_steep)